DOCUMENTO PARA AUTOMATIZAR EL PROCESO DE ETL , DATOS DE YELLOW TAXIS

ETL del archivo yellowtaxis_final.csv  -- PARA GOOGLE CLOUD

Librerias a importar

In [1]:
# Librerias para el proceso de Webscraping

import re
from colorama import Fore
import requests
import urllib.request

In [2]:
# Librerias del proceso de ETL y reduccion

import os
import pandas as pd
import numpy as np
from datetime import datetime
import pyarrow.parquet as pq

----

In [3]:
path = "Data/yellowtaxis_final.csv"

data = pd.read_csv(path)

In [10]:
yellow_taxis = data.copy()

----

# WEBSCRAPING

# COLOCAR WEBCRAPING PARA EL GOOGLE CLOUD

In [ ]:
#La web la que queremos scrappear
website = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
#Hacemos una peticion al sitio web que seleccionamos
resultado = requests.get(website)
#Convertimos el resultado a texto
content = resultado.text
#Para hacer webscrapping tengo que encontrar en el codigo de html de la pagina un patron en comun que tengan
#las cosas que quiero extraer [\w-]*.* detecta todo lo que va despues de lo que asignamos en patron
patron = r'<a\s+href="(.*?/trip-data/yellow_tripdata[^"]*)"'
#Queremos buscar con la expresion regular del patron dentro de content
maquinas = re.findall(patron, str(content))
#En caso de haber duplicados debemos hacer lo siguiente:
#sin_duplicados = list(set(maquinas_repetidas))
# Debemos dejar solamente el nombre ejemplo tenemos 'data-answer="faq2023" class="faq-questions" y debe estar solo faq2023


maquinas_final = []
nombre_archivos = []
for i in maquinas:
    if int(i[64:68]) > 2018 and int(i[64:68]) <= 2023:
        maquinas_final.append(i)
        nombre_archivos.append(i[48:79])
        print(i)

nombre_archivos

In [ ]:
# for a, b in zip(maquinas_final, nombre_archivos):
    urllib.request.urlretrieve(a, b)

# Fin del WebScraping

-----
---
---

# Reduccion del data set al 5%

In [9]:
def reducir_archivo_taxis(carpeta_origen, carpeta_destino):
    # Carpeta_origen, es la ruta de dicha carpeta
    # Carpeta_destino, es la ruta de la carpeta a ser guardado

    # Obtener la lista de archivos Parquet en la carpeta de origen
    archivos_parquet = [archivo for archivo in os.listdir(carpeta_origen) if archivo.endswith('.parquet')]

    # Iterar sobre cada archivo Parquet
    for archivo in archivos_parquet:
        ruta_parquet = os.path.join(carpeta_origen, archivo)

        # Leer el archivo Parquet y convertirlo a DataFrame
        tabla_parquet = pq.read_table(ruta_parquet)
        dataframe = tabla_parquet.to_pandas()

        # Realizar las operaciones necesarias en el DataFrame
        dataframe = dataframe.sample(frac=0.05, random_state=123).reset_index(drop=True)

        # Crear la ruta para el archivo de salida en la carpeta destino
        ruta_parquet_destino = os.path.join(carpeta_destino, archivo)

        # Guardar el DataFrame
        dataframe.to_parquet(ruta_parquet_destino, index=False)

In [10]:
carpeta_origen = 'Data/taxis'
carpeta_destino = 'Data/taxis/Reducido' 
reducir_archivo_taxis(carpeta_origen, carpeta_destino)

----

# Generar la union de los data sets

----
---
---

# ETL DEL DATA SET DE YELLOW TAXIS

In [12]:
# Convertir las columnas en tipo de dato str a fecha y hora

yellow_taxis["tpep_pickup_datetime"] = pd.to_datetime(yellow_taxis["tpep_pickup_datetime"], format='%Y-%m-%d %H:%M:%S')
yellow_taxis["tpep_dropoff_datetime"] = pd.to_datetime(yellow_taxis["tpep_dropoff_datetime"], format='%Y-%m-%d %H:%M:%S')

# Crea Columna nueva
yellow_taxis["trip_datatime"] = yellow_taxis["tpep_dropoff_datetime"] - yellow_taxis["tpep_pickup_datetime"]

----

Columna VendorID <br>
Segun el diccionario no existen ID mayor a 2 <br>
Se dejan por que no aporta nada

----

Columna tpep_pickup_datetime <br>
Elimino los campos que son menor al año 2019

In [13]:
# Filtrar los campos donde el año sea menor a 2019 y eliminar esas filas del DataFrame
yellow_taxis = yellow_taxis[yellow_taxis["tpep_pickup_datetime"].dt.year >= 2019]

Columna tpep_dropoff_datetime <br>
Elimino los campos que son menor al año 2019

In [14]:
# Filtrar los campos donde el año sea menor a 2019 y eliminar esas filas del DataFrame
yellow_taxis = yellow_taxis[yellow_taxis["tpep_dropoff_datetime"].dt.year >= 2019]

In [15]:
# Dividir la columna tpep_pickup_datetime en fecha y hora
yellow_taxis['pickup_date'] = pd.to_datetime(yellow_taxis['tpep_pickup_datetime']).dt.date
yellow_taxis['pickup_time'] = pd.to_datetime(yellow_taxis['tpep_pickup_datetime']).dt.time

# Dividir la columna tpep_dropoff_datetime en fecha y hora
yellow_taxis['dropoff_date'] = pd.to_datetime(yellow_taxis['tpep_dropoff_datetime']).dt.date
yellow_taxis['dropoff_time'] = pd.to_datetime(yellow_taxis['tpep_dropoff_datetime']).dt.time

# Dividir la columna tpep_dropoff_datetime en fecha y hora
yellow_taxis['trip_daystime'] = pd.to_datetime(yellow_taxis['tpep_dropoff_datetime']).dt.date
yellow_taxis['trip_datatime'] = pd.to_datetime(yellow_taxis['tpep_dropoff_datetime']).dt.time


# Eliminar las columnas originales si así lo deseas
yellow_taxis.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_daystime' ], axis=1, inplace=True)

In [17]:
yellow_taxis.head()

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,...,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Airport_fee,trip_datatime,pickup_date,pickup_time,dropoff_date,dropoff_time
0,1,1.0,4.40,1.0,N,239,234,1,18.0,2.5,...,0.3,25.56,2.5,NaN,NaN,08:44:03,2019-04-18,08:20:22,2019-04-18,08:44:03
1,2,3.0,2.96,1.0,N,161,79,1,11.0,0.5,...,0.3,17.76,2.5,NaN,NaN,21:33:29,2019-04-15,21:21:29,2019-04-15,21:33:29
2,2,3.0,0.96,1.0,N,114,125,1,8.0,0.0,...,0.3,14.12,2.5,NaN,NaN,14:46:15,2019-04-05,14:35:47,2019-04-05,14:46:15
3,2,1.0,0.91,1.0,N,239,238,1,5.5,1.0,...,0.3,11.76,2.5,NaN,NaN,19:54:02,2019-04-01,19:49:33,2019-04-01,19:54:02
4,1,1.0,1.50,1.0,N,68,142,2,9.0,3.0,...,0.3,12.80,2.5,NaN,NaN,20:14:37,2019-04-27,20:03:50,2019-04-27,20:14:37


---

Columna passenger_count <br>
Hay muchos viajes con mas de 4 pasajeros, seran los vehiculos tipo SUV o parecidos con mas capacidad. <br>
No hay outliers

In [7]:
# Eliminar filas con valores nulos en la columna 'passenger_count'
yellow_taxis = yellow_taxis.dropna(subset=['passenger_count'])

La columna con pasajeros igual a CERO, no tiene sentido. <br>
Le asigno valores , interpolando con los datos existentes, como dato tipo int

In [8]:
# Convertir 0s a NaNs en la columna 'passenger_count' utilizando .loc[]
yellow_taxis.loc[yellow_taxis['passenger_count'] == 0, 'passenger_count'] = np.nan

In [9]:
# Interpolar valores nulos en la columna 'passenger_count'
yellow_taxis.loc[:, 'passenger_count'] = yellow_taxis['passenger_count'].interpolate(method='linear')

# Convertir la columna 'passenger_count' a tipo entero
yellow_taxis['passenger_count'] = yellow_taxis['passenger_count'].astype(int)


----
Columna trip_distance, no se le hace nada

Columna RateCode ID

Hay datos que no se identifican con ninguno del diccionario! <br>

---

Columna store_and_fwd_flag <br>
Se elimina por poco utilidad

In [10]:
yellow_taxis.drop('store_and_fwd_flag', axis=1, inplace=True)

----

Columna Fare amount <br> 
Tiene outliers menores a 0 y algunos valores muy altos <br>
Los valores mayores a 100.000 se borran y los negativos tmb <br>
Filtro todas estas variables con columna "total amount"

Columna MTA_tax <br>
Hay valores menores a 0 <br>
Lo trabajo con columna "total amount"

----

Columna improvement_surcharge <br>
Oscila entre -1 a 1

---

Columna total_amount

In [13]:
filtro_total_amount_mayores = yellow_taxis['total_amount'] > 10000
# total_amount_mayores = yellow_taxis[filtro_total_amount_mayores]

# Eliminar las filas con valores mayor a 100000
yellow_taxis = yellow_taxis.drop(yellow_taxis[filtro_total_amount_mayores].index)

In [14]:
# Eliminar las filas que tienen total amount negativos
yellow_taxis = yellow_taxis.drop(yellow_taxis[yellow_taxis['total_amount'] < 0].index)

----

----
----
----
# VER !!!


# hacer la division de fechas y horas en las columnas

# Fin del ETL

----

# Carga de los datos a BIG QUERY

---
---
---

----
